Since my major question focuses on gathering data from the BRICS nations, one of the initial problems faced was trying to find reliable sources to same basal columns for the five countries of Brazil, India, China, South Africa and Russia. 

Whilst finding the data for similar columns, one of the issues I had faced for, for instance, if China had information about Education, which was also one of the parameters I wanted to explore, South Africa's education did not have cut offs in data collection for primary, secondary and tertiary schools. If Adjusted National was calculated for Brazil, India's Net National Income wasn't properly adjusted. Hence, usage of API was indeed the best step forward as it helped calculated all the data points on an equal scale. 


# Data Source 

Since the data is that is majorly required is numerical data. I am retrieving data through API's and by downloading data from World Bank.

In [22]:
# from fred api
from fredapi import Fred
import pandas as pd
from IPython.display import display
from pprint import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import nltk
from sklearn.feature_extraction.text import CountVectorizer

## Quantitative Data

The data required for this project is a culmunation of macroeconomic and fiscal indicators of all the five emerging economics for which individual data has to be retrived using various sources.

### Python API - FRED API

The Python API used for this project is the FRED API. The FRED-API is a web service that allows developers to create programmes and construct apps that retrieve statistical information about the economy from the FRED and ALFRED websites hosted by the Federal Reserve Bank of St. Louis' Economic Research Division using Python. 

Since we are using data of the 5 countries, it can be interesting to compare it with a hegemonic and the most powerful country in the world, The United States of America so using the FRED-API, you can clone few macroeconomic indicators from here. 

The FRED API has been called and the series IDs have been added to the data.

In [18]:

# start and end date
start_date = '1973'
end_date = '2023'
fred = Fred(api_key='10fbe66f8f62ad7f44097cca867bf01f')

Trying to retrive the second part of the US data which is useful during the exploratory data analysis

In [19]:
# series required
series_id_1 = 'GDP' # for GDP US
series_id_2='GDPC1' # for Real Gross GDP
series_id_3='W207RC1Q156SBEA' # Adjusted Saving (% of GNI)
series_id_4='USAB6BLTT02STSAQ' # Current account balance (% of GDP)
series_id_5='A019RE1Q156NBEA' # Export of Goods and Services as percentage of GDP
series_id_6='MKTGNIUSA646NWDB' # GNI

# Fetch the data
gdp_u = fred.get_series(series_id_1, start_date, end_date)
realgdp_u = fred.get_series(series_id_2, start_date, end_date)
adjsavings_u = fred.get_series(series_id_3, start_date, end_date)
currentaccbalance_u = fred.get_series(series_id_4, start_date, end_date)
exportsofgoods_u = fred.get_series(series_id_5, start_date, end_date)
gni_u= fred.get_series(series_id_6, start_date, end_date)

In [20]:
print(gni_u.head(4))

1973-01-01    1.43e+12
1974-01-01    1.55e+12
1975-01-01    1.68e+12
1976-01-01    1.87e+12
dtype: float64


Since the data has been fetched like this using the 'get_series' function, the data was on basis of quarterly collected which is again regrouped and after pre-processing, this is how the data looks like.

In [21]:
df= pd.read_csv('../data/raw_fred_data_us.csv')

display(df.head(15))

,Unnamed: 0,Year,GDP,Real GDP,Adjusted Savings,Current Account Balance,Exports of Goods and Services,GNI
0,0,1973,1425.38,6106.37,10.93,0.49,0.28,1.43e+12
1,1,1974,1545.24,6073.36,9.20,0.13,-0.05,1.55e+12
2,2,1975,1684.90,6060.88,6.50,1.08,0.95,1.68e+12
3,3,1976,1873.41,6387.44,7.45,0.23,-0.05,1.87e+12
4,4,1977,2081.82,6682.80,8.12,-0.68,-1.12,2.08e+12
5,5,1978,2351.60,7052.71,9.38,-0.66,-1.12,2.35e+12
6,6,1979,2627.33,7276.00,9.20,-0.01,-0.88,2.61e+12
7,7,1980,2857.31,7257.32,7.05,0.07,-0.45,2.85e+12
8,8,1981,3207.04,7441.48,8.00,0.16,-0.38,3.20e+12
9,9,1982,3343.79,7307.31,5.62,-0.16,-0.60,3.37e+12


## Textual Data

Understanding textual data for BRICS is critical since it provides significant insights into each member country's socioeconomic and geopolitical landscape. Textual data analysis, such as documents, news stories, and social media, aids in uncovering hidden trends, identifying emerging difficulties, and gauging overall attitude within each country. This detailed understanding is essential for developing comprehensive plans, encouraging collaboration, and resolving common concerns.

### Python API

Understanding the perspective of people on the emergence of BRICS is just as crucial, hence we need to ensure that the public opinions have also been catered and taken into consideration. For this, the REDDIT API for the subreddits have been used. REDDIT API with limited access helps us extract the subreddits for that specific day with the ragin subreddits. The Python PRAW has been used to retrieve the data.

In [24]:
import praw
from praw.models import MoreComments

user_agent='Sheeba_Economic'
reddit= praw.Reddit(
    client_id="rszab0dBkcT7NdZr6MftpQ",
    client_secret="mBFjTSN3YXoSimkU7eZ1H0SoD-f2Yw",
    user_agent=user_agent)

posts = []
for submission in reddit.subreddit("Brics").top("all"):
    for top_level_comment in submission.comments:
        if isinstance(top_level_comment, praw.models.MoreComments):
            continue
        posts.append(top_level_comment.body)
        
df = pd.DataFrame(posts, columns=["body"])

/var/folders/cm/1bq_zvw92w99j_5d1p5jq5v40000gn/T/ipykernel_25910/988797959.py:11: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  for submission in reddit.subreddit("Brics").top("all"):


In [28]:
display(df.head(15))


,body
0,Burn him with his flag 🤬
1,This plan is to kill the Petro Dollar run and ...
2,It’s difficult to know what’s going to come ou...
3,There is a lot to unpack as to what BRICS woul...
4,"Nice that you ""feel"" that way. That's what BRI..."
5,"Seems like there well ahead of schedule, they ..."
6,"Yeah, precisely. I fully explained this today ..."
7,I watched a lecture at school that predicted t...
8,"I think, and I hope, that it happens sooner, r..."
9,It's supposed to happen in about a week hold o...


Here, we can see that the data has a lot of cleaning to do, but this data in gist will help to understand the opinions of people since it has been pulled from the REDDIT API.